<a href="https://colab.research.google.com/github/Erfandarzi/JAFarGAN/blob/master/Federated_Cifar_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
!pip install syft==0.2.3
!pip install pillow==6.2.1

In [4]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

In [5]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [6]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!





In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import time 

import syft as sy  # <-- NEW: import the Pysyft library

class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 2500
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = True
        self.seed = 1
        self.log_interval = 200
        self.save_model = False
        self.accuracy = 98 

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()
use_cuda = 0 

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

def load_data():
    
    '''<--Load CIFAR dataset from torch vision module distribute to workers using PySyft's Federated Data loader'''
    

    federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.CIFAR10('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

    test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)
    
    return federated_train_loader,test_loader


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = int(100. * correct / len(test_loader.dataset))

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return accuracy 



#<--Load federated training data and test data
federated_train_loader,test_loader=load_data()

#<--Create Neural Network model instance
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) #<--TODO momentum is not supported at the moment

#<--Train Neural network and validate with test set after completion of training every epoch
for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    accuracy = test(args, model, device, test_loader)
    if int(accuracy) >= args.accuracy:
        time2 = time.time()
        print("total time elapsed is ", time2 - time1)
        break


if (args.save_model):
    torch.save(model.state_dict(), "cifar10_cnn.pt")

Extracting ../data/cifar-10-python.tar.gz to ../data

Train Epoch: 1 [0/50048 (0%)]	Loss: 2.310682
Train Epoch: 1 [12800/50048 (26%)]	Loss: 2.301998
Train Epoch: 1 [25600/50048 (51%)]	Loss: 2.293859
Train Epoch: 1 [38400/50048 (77%)]	Loss: 2.298970

Test set: Average loss: 2.2804, Accuracy: 1528/10000 (15%)

Train Epoch: 2 [0/50048 (0%)]	Loss: 2.285189
Train Epoch: 2 [12800/50048 (26%)]	Loss: 2.214281
Train Epoch: 2 [25600/50048 (51%)]	Loss: 2.174301
Train Epoch: 2 [38400/50048 (77%)]	Loss: 1.991645

Test set: Average loss: 1.9646, Accuracy: 2947/10000 (29%)

Train Epoch: 3 [0/50048 (0%)]	Loss: 1.924952
Train Epoch: 3 [12800/50048 (26%)]	Loss: 1.949415
Train Epoch: 3 [25600/50048 (51%)]	Loss: 1.895669
Train Epoch: 3 [38400/50048 (77%)]	Loss: 1.820386

Test set: Average loss: 1.7785, Accuracy: 3498/10000 (35%)

Train Epoch: 4 [0/50048 (0%)]	Loss: 1.611385
Train Epoch: 4 [12800/50048 (26%)]	Loss: 1.611142
Train Epoch: 4 [25600/50048 (51%)]	Loss: 1.711248
Train Epoch: 4 [38400/50048 (77%)

Exception ignored in: <bound method ObjectPointer.__del__ of [PointerTensor | me:33548330239 -> bob:25896848038]>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/syft/generic/pointers/object_pointer.py", line 344, in __del__
    self.owner.send_msg(ForceObjectDeleteMessage(self.id_at_location), self.location)
  File "/usr/local/lib/python3.6/dist-packages/syft/workers/base.py", line 282, in send_msg
    bin_response = self._send_msg(bin_message, location)
  File "/usr/local/lib/python3.6/dist-packages/syft/workers/virtual.py", line 15, in _send_msg
    return location._recv_msg(message)
  File "/usr/local/lib/python3.6/dist-packages/syft/workers/virtual.py", line 19, in _recv_msg
    return self.recv_msg(message)
  File "/usr/local/lib/python3.6/dist-packages/syft/workers/base.py", line 309, in recv_msg
    msg = sy.serde.deserialize(bin_message, worker=self)
  File "/usr/local/lib/python3.6/dist-packages/syft/serde/serde.py", line 46, in deserialize
 

Train Epoch: 27 [12800/50048 (26%)]	Loss: 0.724599
Train Epoch: 27 [25600/50048 (51%)]	Loss: 0.866861


KeyboardInterrupt: ignored

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import syft as sy 
import random 


# In[15]:


class Arguments():
    def __init__(self, epochs=1):
        self.batch_size = 128
        self.test_batch_size = 1000
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = True
        self.seed = 200226097 ## TODO change seed to your studentID inside the class Arguments (line 17)
        self.log_interval = 30
        self.save_model = False

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


def train(args, model, device, federated_train_loader, optimizer, epoch, participates):
    model.train()  # <-- initial training
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        if target.location.id in participates:
            model.send(data.location) # <-- NEW: send the model to the right location
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            model.get() # <-- NEW: get the model back
            if batch_idx % args.log_interval == 0:
                loss = loss.get() # <-- NEW: get the loss back
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                    100. * batch_idx / len(federated_train_loader), loss.item()))


            
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


### main function
def run(N=1, X=1):
    args = Arguments(epochs = N)
    use_cuda = not args.no_cuda and torch.cuda.is_available()
    torch.manual_seed(args.seed) 
    device = torch.device("cuda" if use_cuda else "cpu")
    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

    hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning

    ## TODO: ----  create 10 node workers  ---- ##
    workers = []
    worker_ids = []
    for i in range(10):
        worker_name = f"worker{i}"
        worker = sy.VirtualWorker(hook, id=worker_name)
        worker_ids.append(worker_name)
        workers.append(worker)
    ##-------------------------------------------

    ## distribute data across nodes
    federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
        datasets.MNIST('./data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))
        .federate((workers)), ##TODO: pass the worker nodes you created here to distribute the data
        batch_size=args.batch_size, shuffle=True, **kwargs)

    ## test dataset is always same at the central server
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)

    ## training models in a federated appraoch
    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr) 

    ## TODO: select a random set of node ids that will be passed to the training function; these nodes will particiapte in the federated learning
    #create node_list 

    node_list = random.sample(worker_ids, k=X)

    ##-------------------------------------------

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, federated_train_loader, optimizer, epoch, node_list ) ## TODO: pass the node_id list like ['node1','node2' ...]
        test(args, model, device, test_loader)

    if (args.save_model):
        torch.save(model.state_dict(), "mnist_cnn.pt")







# In[16]:


print("Problem 2a.")
x_vals = [3, 5, 7, 10]
for x in x_vals:
    print(f"Running with N=3, X={x}")
    run(N=3, X=x)

    


# In[17]:


print("Problem 2b.")
n_vals = [3, 5, 10]
for n in n_vals:
    print(f"Running with X=5, N={n}")
    run(N=n, X=5)


# In[ ]:



Problem 2a.
Running with N=3, X=3


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


Train Epoch: 1 [19200/60032 (32%)]	Loss: 2.263730
Train Epoch: 1 [23040/60032 (38%)]	Loss: 2.097992
